---

## DCFC EVCS POI Candidate Selection - Atlanta

**Atlanta**: Since a larger number of DCFC-type EVCS (Electric Vehicle Charging Stations) are installed in Atlanta compared to other regions, an initial **POI filtering** was conducted to identify potential locations for DCFC installation. Additionally, to further refine the placement of DCFC EVCS, **case-specific optimization** was applied. This process first involved running an optimization for **Level 2 EVCS** installations. The results from this optimization were then used as additional candidates for DCFC installation.

---

### Summary
  
- **DCFC POI Candidate Selection**: Atlanta, using GIS tools, filtering, and optimization of Level 2 EVCS results.  

-  **POI Filtering Results** + **Optimization of Level 2 EVCS** = Final **DCFC EVCS Candidates** for Atlanta.  
<br>
---


# DCFC POI Candidate Selection - Atlanta

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
gpkg_file_path = "Path To your Atlanta Region POI"
result_of_atlanta = 'Path to Optimization Result in Atlanta' 

# 2. Read the GPKG file
poi_gdf = gpd.read_file(gpkg_file_path)
poi2_gdf = gpd.read_file(result_of_atlanta)

# 3. List of categories to filter
selected_category = ['bank', 'supermarket', 'department_store', 'hospital', 'public_building', 'mall', 'town_hall']

# 4. Filter rows from the 'fclass' column that match the selected categories
poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_category)]

poi2_gdf = poi2_gdf[['osm_id', 'fclass', 'geometry']]

merged = pd.concat([poi_gdf, poi2_gdf], ignore_index=True)

merged= merged.drop_duplicates(subset='osm_id')

merged_gdf = gpd.GeoDataFrame(merged, geometry='geometry')

merged_gdf.to_file("atlanta_dcfc_poi_candidate.gpkg", driver="GPKG")

---

## Case 2: Atlanta DCFC EVCS Charger Initial Point Selection

The **Atlanta** area is a key region for the installation of numerous **DCFC** (Direct Current Fast Chargers).  
Since the optimal location of DCFCs can be critical to traffic flow, we use the **road network** to guide the selection of initial points.

### Algorithm:

1. **Map demand values** using the **Motorway** and **Trunkway Demand Map**. Start by selecting POIs adjacent to the roads with the highest demand values based on the following criteria:
2. Find **POI candidates** near roads with high demand values, and ensure that the selected POIs maintain a **minimum distance of 3 km**, which is the minimum service radius for EVCS.
3. Select the specified number of **initial POIs**, repeating the process until the required number of points is reached.
4. If fewer than 5 POIs are selected in a specific area, the process reads the **Demand Map's pixel values** directly to identify high-demand regions and select additional POI candidates.  
<br>  
---

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
import numpy as np
from shapely.geometry import box
from geopy.distance import geodesic

In [ ]:
motorway_gdf = gpd.read_file('Path to Your Road Network_Motorway.gpkg')  # Load motorway road network
trunk_gdf = gpd.read_file('Path to Your Road Network_Trunk.gpkg')  # Load trunk road network
poi_gdf = gpd.read_file('Path to your Atalanta_DCFC_Candidate.gpkg')  # Load candidate POIs for DCFC
geotif = rasterio.open('Path to your Demand Map.tif')  # Load demand map raster
boundary_gdf = gpd.read_file('Path to your Atlanta Polygon.gpkg')  # Load Atlanta boundary polygon

# Coordinate system transformation settings: Target CRS and WGS84 for distance calculations
target_crs = 'EPSG:3857'  # CRS for mapping
wgs84_crs = 'EPSG:4326'  # CRS for geodesic distance calculations (WGS 84)

# Transform the CRS of each dataset to the target CRS (EPSG:3857) if they don't match
if motorway_gdf.crs.to_string() != target_crs:
    motorway_gdf = motorway_gdf.to_crs(target_crs)
if trunk_gdf.crs.to_string() != target_crs:
    trunk_gdf = trunk_gdf.to_crs(target_crs)
if poi_gdf.crs.to_string() != target_crs:
    poi_gdf = poi_gdf.to_crs(target_crs)
if boundary_gdf.crs.to_string() != target_crs:
    boundary_gdf = boundary_gdf.to_crs(target_crs)

# Function to extract pixel values from the demand map for given geometries (roads)
def get_pixel_values(geometries, geotif):
    pixel_values = []
    for geom in geometries:
        # Create a mask from the geometry and extract pixel data from the demand map
        mask = geometry_mask([geom], transform=geotif.transform, invert=True, out_shape=(geotif.height, geotif.width))
        pixel_data = geotif.read(1, masked=True)[mask]  # Read the demand values where the geometry intersects
        if pixel_data.size > 0:
            pixel_values.append((geom, np.max(pixel_data)))  # Append the highest demand value for the geometry
        else:
            pixel_values.append((geom, None))  # Append None if no demand value is available
    return pixel_values

# Function to calculate the geodesic distance between two points in kilometers
def distance_in_km(point1, point2):
    return geodesic((point1.y, point1.x), (point2.y, point2.x)).km

# Function to check if a new POI is at least 3km away from existing selected POIs
def is_far_enough(selected_pois, new_poi_geom, min_distance_km=3):
    # Transform both selected POIs and new POI to WGS84 for distance calculation
    selected_pois_wgs84 = selected_pois.to_crs(wgs84_crs)
    new_poi_gdf = gpd.GeoDataFrame(geometry=[new_poi_geom], crs=selected_pois.crs)  # Ensure CRS consistency
    new_poi_wgs84 = new_poi_gdf.to_crs(wgs84_crs)

    # Check the geodesic distance between the new POI and each selected POI
    for poi in selected_pois_wgs84.itertuples():
        if distance_in_km(poi.geometry, new_poi_wgs84.geometry.iloc[0]) < min_distance_km:
            return False  # If the distance is less than the minimum distance (3km), return False
    return True  # If all distances are valid, return True

# Select 3 initial points for each region, ensuring a minimum distance of 3km between POIs
initial_candidates = []

for fid in boundary_gdf['fid'].unique():
    # Filter the boundary for each region (by fid)
    region_boundary = boundary_gdf[boundary_gdf['fid'] == fid]
    
    # Select motorway and trunk roads within the region boundary
    motorway_in_region = motorway_gdf[motorway_gdf.intersects(region_boundary.unary_union)]
    trunk_in_region = trunk_gdf[trunk_gdf.intersects(region_boundary.unary_union)]
    
    # Combine motorway and trunk roads into one GeoDataFrame
    combined_gdf = gpd.GeoDataFrame(pd.concat([motorway_in_region, trunk_in_region], ignore_index=True))
    
    # Get demand values for the combined road geometries
    combined_pixel_values = get_pixel_values(combined_gdf.geometry, geotif)
    combined_pixel_values = [v for v in combined_pixel_values if v[1] is not None]  # Filter out None values
    combined_pixel_values.sort(key=lambda x: x[1], reverse=True)  # Sort by demand value (descending)

    # Initialize an empty GeoDataFrame for storing selected POIs in the region
    region_candidates = gpd.GeoDataFrame(columns=poi_gdf.columns, crs=target_crs)

    # Loop through each road segment, selecting POIs based on proximity and demand
    for geom, _ in combined_pixel_values:
        candidates_in_pixel = poi_gdf[poi_gdf.intersects(geom)]  # Find POIs intersecting the road geometry
        for _, candidate_poi in candidates_in_pixel.iterrows():
            candidate_gdf = gpd.GeoDataFrame([candidate_poi], crs=poi_gdf.crs)  # Create GeoDataFrame for the candidate
            if len(region_candidates) > 0 and not is_far_enough(region_candidates, candidate_gdf.geometry.iloc[0]):
                continue  # Skip the POI if it's too close to already selected POIs
            region_candidates = pd.concat([region_candidates, candidate_gdf], ignore_index=True)  # Add the POI
            if len(region_candidates) >= 5:
                break  # Stop if 5 POIs have been selected for the region
        if len(region_candidates) >= 5:
            break  # Stop if 5 POIs have been selected for the region

    # If fewer than 5 POIs are selected, use demand map pixel values directly
    if len(region_candidates) < 5:
        pixel_data = geotif.read(1)  # Read demand map data
        sorted_indices = np.unravel_index(np.argsort(pixel_data, axis=None)[::-1], pixel_data.shape)  # Sort by demand
        
        # Loop through the highest demand pixels and find additional POIs
        for y, x in zip(sorted_indices[0], sorted_indices[1]):
            geom = box(*geotif.xy(y, x), *geotif.xy(y + 1, x + 1))  # Create a geometry for the pixel
            if region_boundary.unary_union.intersects(geom):  # Check if the pixel is within the region
                candidates_in_pixel = poi_gdf[poi_gdf.intersects(geom)]  # Find POIs in the pixel
                for _, candidate_poi in candidates_in_pixel.iterrows():
                    candidate_gdf = gpd.GeoDataFrame([candidate_poi], crs=poi_gdf.crs)  # Create GeoDataFrame for the POI
                    if len(region_candidates) > 0 and not is_far_enough(region_candidates, candidate_gdf.geometry.iloc[0]):
                        continue  # Skip the POI if it's too close to already selected POIs
                    region_candidates = pd.concat([region_candidates, candidate_gdf], ignore_index=True)
                    if len(region_candidates) >= 5:
                        break  # Stop if 5 POIs have been selected
            if len(region_candidates) >= 5:
                break  # Stop if 5 POIs have been selected

    initial_candidates.append(region_candidates)  # Add the selected candidates for the region

# Save the selected POIs to a GeoPackage file
if initial_candidates:
    initial_candidates_gdf = gpd.GeoDataFrame(pd.concat(initial_candidates, ignore_index=True), crs=target_crs)
    initial_candidates_gdf.to_file('atlanta_dcfc_initial.gpkg', driver="GPKG")  # Save to GPKG
    print(f"Total selected POIs: {len(initial_candidates_gdf)}")  # Print the total number of selected POIs
else:
    print("No POIs were selected.")  # Print message if no POIs were selected


---

## Optimization

In [ ]:
import geopandas as gpd

from src import greedy_optimization
from src import setup_logging, merge_gpkg_files


setup_logging()

gpkg_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/suburban_trip_greedy.gpkg"
tif_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/demand_map_500.tif"
poi_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/Suburban_POI_Candidate.gpkg"
output_path = "./results/"

# Read input files
polygons = gpd.read_file(gpkg_file)
poi_gdf = gpd.read_file(poi_file)

for _, polygon in polygons.iterrows(): 
    greedy_optimization(
        polygon, 
        tif_file, 
        poi_gdf, 
        capture_range=3000, 
        bandwidth=1000, 
        constraints=(1, None), 
        output_path=output_path,
        save_intermediate=True
    )

# # Merge all GPKG files in the output path
# merge_gpkg_files(output_path, output_file_name="suburban.gpkg")